# <p style="text-align: center;">**Oceananigans.jl**<br><br><del>Massively parallel</del> non-hydrostatic ocean modeling <del>on GPUs</del> in Julia</p>
## <p style="text-align: center;">**GitHub repo**: [ali-ramadhan/Oceananigans.jl](https://github.com/ali-ramadhan/Oceananigans.jl)<br><br>**This notebook**: [ali-ramadhan/random-jupyter-notebooks](https://github.com/ali-ramadhan/random-jupyter-notebooks)</p>

---
## **Acknowledgements**
* While I did write the code, I did not do this alone!
* **EAPS**: Chris Hill, Jean-Michel Campin, John Marshall, Greg Wagner, Mukund Gupta.
* **Julia Lab**: Valentin Churavy, Peter Ahrens, Shashi Gowda, Sung Woo Jeong, Ranjan Anantharaman, Alan Edelman.
---

### **Julia Packages used**: `FFTW, CUDAnative, CUDAdrv, CuArrays, DistributedArrays, PyPlot, Makie, Interact, Documenter, BenchmarkTools`
---

## **Navier-Stokes equations in a non-inertial reference frame**
### $$\rho \frac{D \mathbf{u}}{D t}  = -\nabla p + \mu \, \nabla^2 \mathbf{u} + \tfrac13 \mu \, \nabla (\nabla\cdot\mathbf{u}) + \rho\mathbf{g} - \rho \left( 2\mathbf{\Omega} \times \mathbf{u} + \mathbf{\Omega} \times \mathbf{\Omega} \times \mathbf{x} + \frac{d \mathbf{U}}{dt} + \frac{d\mathbf{\Omega}}{dt} \times \mathbf{x} \right)$$
### where $\displaystyle \frac{D}{Dt} = \frac{\partial}{\partial t} + \mathbf{u} \cdot \nabla$ is the material derivative, $\mathbf{g}$ is the acceleration due to gravity, $\rho$ is the fluid density, $\mathbf{x}$ is the position vector, $\mathbf{u} = (u,v,w)$ is the fluid flow velocity, $\mathbf{U}$ is the frame translation velocity,  $\mu$ is the viscosity, and $\Omega$ is the frame angular velocity. Kind of like $F=ma$.
* ### _"Hyperbolic in time"_ when advection dominated. _"Parabolic in time"_ when diffusion dominated. _"Elliptic in space"_.
---

## **Governing equations for non-hydrostatic ocean modeling**
### To be more exact: _Rotating, incompressible, Boussinesq equations of motion_
## Horizontal momentum: $\displaystyle \frac{\partial \mathbf{u}_h}{\partial t} = \mathbf{G}_{uh} - \nabla_h p$
## Vertical momentum: $\displaystyle \frac{\partial w}{\partial t} = G_w - \frac{\partial p}{\partial z}$
## Continuity equation (mass conservation): $\displaystyle \nabla \cdot \mathbf{u} = 0$
## Temperature: $\displaystyle \frac{\partial T}{\partial t} = G_T$
## Salinity: $\displaystyle \frac{\partial S}{\partial t} = G_S$
## Equation of state: $\displaystyle \rho = \rho(T,S,p)$
where $\mathbf{u} = (\mathbf{v}_h, w) = (u,v,w)$ is the velocity, $\mathbf{v}_h = (u,v)$ is the horizontal velocity, $\nabla = (\partial_x, \partial_y, \partial_z)$ is the del operator, $\nabla_h = (\partial_x, \partial_y)$ is the horizontal del operator. The equation of state for seawater gives the density $\rho$ in terms of $T$, $S$, and $p$. $\mathbf{G}_u = (\mathbf{G}_{uh}, G_w) = (G_u, G_v, G_w)$ represents inertial, Coriolis, gravitational, forcing, and dissipation terms.
---

## **Source terms**
\begin{align}
    G_u &= -\mathbf{u} \cdot \nabla u + fv + \nabla\cdotp(\nu \nabla u) + F_u \\
    G_v &= -\mathbf{u} \cdot \nabla v - fu + \nabla\cdotp(\nu \nabla v) + F_v \\
    G_w &= -\mathbf{u} \cdot \nabla w -g \frac{\delta \rho}{\rho_0} + \nabla\cdotp(\nu \nabla w) + F_w \\
    G_T &= -\nabla \cdot (\mathbf{u} T) + \nabla\cdotp(\kappa \nabla T) + F_T \\
    G_S &= -\nabla \cdot (\mathbf{u} S) + \nabla\cdotp(\kappa \nabla S) + F_S
\end{align}
---

In [ ]:
Gᵘⁿ = -ũ∇u(uⁿ, vⁿ, wⁿ) .+ f .* avgʸc2f(avgˣf2c(vⁿ)) .- (1/Δx) .* δˣc2f(pʰʸ′ ./ ρ₀) .+ 𝜈ʰ∇²u(uⁿ) .+ Fᵘ
Gᵛⁿ = -ũ∇v(uⁿ, vⁿ, wⁿ) .- f .* avgˣc2f(avgʸf2c(uⁿ)) .- (1/Δy) .* δʸc2f(pʰʸ′ ./ ρ₀) .+ 𝜈ʰ∇²v(vⁿ) .+ Fᵛ
Gʷⁿ = -ũ∇w(uⁿ, vⁿ, wⁿ)                                                             .+ 𝜈ᵛ∇²w(wⁿ) .+ Fʷ
Gᵀⁿ = -div_flux_f2c(uⁿ, vⁿ, wⁿ, Tⁿ) + κ∇²(Tⁿ) + Fᵀ
Gˢⁿ = -div_flux_f2c(uⁿ, vⁿ, wⁿ, Sⁿ) + κ∇²(Sⁿ) + Fˢ

### where $f = 2\Omega\sin\phi$ is the Coriolis frequency, $\Omega$ is the rotation rate of the Earth, $\phi$ is the latitude, $g$ is the acceleration due to gravity, $\delta\rho = \rho - \rho_0$ is the deviation of the density from that of a resting ocean, and $\rho_0$ is some reference density.

## **Spatial discretization and finite volumes**
![](img/cell.png)
![](img/cgrid.png)
---

## **Time stepping**
### Two-step Adams-Bashforth (midpoint) rule for source terms
## $$\mathbf{G}^{n+1/2} = \left( \frac{3}{2} + \chi \right) \mathbf{G}^n - \left( \frac{1}{2} + \chi \right) \mathbf{G}^{n-1}$$

In [ ]:
@. begin
    Gᵘⁿ⁺ʰ = (3/2 + χ)*Gᵘⁿ - (1/2 + χ)*Gᵘⁿ⁻¹
    Gᵛⁿ⁺ʰ = (3/2 + χ)*Gᵛⁿ - (1/2 + χ)*Gᵛⁿ⁻¹
    Gʷⁿ⁺ʰ = (3/2 + χ)*Gʷⁿ - (1/2 + χ)*Gʷⁿ⁻¹
    Gᵀⁿ⁺ʰ = (3/2 + χ)*Gᵀⁿ - (1/2 + χ)*Gᵀⁿ⁻¹
    Gˢⁿ⁺ʰ = (3/2 + χ)*Gˢⁿ - (1/2 + χ)*Gˢⁿ⁻¹
end

### Explicit time stepping for $u,v,w,T,S$ fields
\begin{align}
  \frac{\mathbf{v}_h^{n+1} - \mathbf{v}_h^n}{\Delta t} &= \mathbf{G}_{vh}^{n+1/2} - \frac{1}{\rho_0}\nabla_h (p_S + p_{HY} + qp_{NH})^{n+1/2} \\
  \frac{w^{n+1} - w^n}{\Delta t} &= \hat{G}_w^{n+1/2} - \frac{\partial p_{NH}^{n+1/2}}{\partial z} \\
  \frac{1}{\Delta t} \left[ \begin{pmatrix}S \\ T\end{pmatrix}^{n+1} - \begin{pmatrix}S \\ T\end{pmatrix}^n \right] &= \mathbf{G}^{n+1/2}_{(S,T)}
\end{align}
---

In [ ]:
uⁿ .= uⁿ .+ ( Gᵘⁿ⁺ʰ .- (1/Δx) .* δˣc2f(pⁿʰ⁺ˢ) ) .* Δt
vⁿ .= vⁿ .+ ( Gᵛⁿ⁺ʰ .- (1/Δy) .* δʸc2f(pⁿʰ⁺ˢ) ) .* Δt
wⁿ .= wⁿ .+ ( Gʷⁿ⁺ʰ .- (1/Δz) .* δᶻc2f(pⁿʰ⁺ˢ) ) .* Δt
@. Sⁿ = Sⁿ + (Gˢⁿ⁺ʰ * Δt)
@. Tⁿ = Tⁿ + (Gᵀⁿ⁺ʰ * Δt)

## **Discretized operators**

### Difference operators $\delta_x$, $\delta_y$, $\delta_z$
### $$\delta_x(f) = f_E - f_W, \quad\quad \delta_y(f) = f_N - f_S, \quad\quad \delta_z(f) = f_T - f_B$$

In [ ]:
@inline incmod1(a, n) = a == n ? one(a) : a + 1
@inline decmod1(a, n) = a == 1 ? n : a - 1

# Example: δx operator for cell-centered difference.

function δˣc2f(f)
    Nx, Ny, Nz = size(f)
    δf = zeros(Nx, Ny, Nz)
    for k in 1:Nz, j in 1:Ny, i in 1:Nx
        δf[i, j, k] =  f[i, j, k] - f[decmod1(i,Nx), j, k]
    end
    δf
end

function δx!(g::RegularCartesianGrid, f::CellField, δxf::FaceField)
    for k in 1:g.Nz, j in 1:g.Ny, i in 1:g.Nx
        @inbounds δxf[i, j, k] =  f[i, j, k] - f[decmod1(i, g.Nx), j, k]
    end
    nothing
end

### Averaging operators ($\overline{\;\cdotp\;}^x$, $\overline{\;\cdotp\;}^y$, $\overline{\;\cdotp\;}^z$)
### $$\overline{\;f\;}^x = \frac{f_E + f_W}{2}, \quad\quad \overline{\;f\;}^y = \frac{f_N + f_S}{2}, \quad\quad \overline{\;f\;}^z = \frac{f_T + f_B}{2}$$

In [ ]:
# Example: z averaging operator for cell-centered difference.

function avgᶻc2f(f)
    Nx, Ny, Nz = size(f)
    δf = zeros(Nx, Ny, Nz)
    for k in 2:Nz, j in 1:Ny, i in 1:Nx
        δf[i, j, k] =  (f[i, j, k] + f[i, j, k-1]) / 2
    end
    @. δf[:, :, 1] = 0
    δf
end

function avgz!(g::RegularCartesianGrid, f::CellField, favgz::FaceField)
    for k in 2:g.Nz, j in 1:g.Ny, i in 1:g.Nx
        @inbounds favgz.data[i, j, k] =  (f.data[i, j, k] + f.data[i, j, k-1]) / 2
    end
    @. favgz.data[:, :, 1] = 0
    nothing
end

### Divergence $\nabla \cdotp \mathbf{f}$

## $$\nabla \cdot \mathbf{f} = \frac{1}{V} \left[ \delta_x (A_x f_x)  + \delta_y (A_y f_y) + \delta_z (A_z f_z) \right]$$

In [ ]:
# Example: divergence operator for face-centered fields.

function div_f2c(fˣ, fʸ, fᶻ)
    (1/V) * ( δˣf2c(Aˣ .* fˣ) + δʸf2c(Aʸ .* fʸ) + δᶻf2c(Aᶻ .* fᶻ) )
end

function div!(g::RegularCartesianGrid,
              fx::FaceFieldX, fy::FaceFieldY, fz::FaceFieldZ, div::CellField,
              tmp::TemporaryFields)

    δxfx, δyfy, δzfz = tmp.fC1, tmp.fC2, tmp.fC3

    δx!(g, fx, δxfx)
    δy!(g, fy, δyfy)
    δz!(g, fz, δzfz)

    @. div.data = (1/g.V) * (g.Ax * δxfx.data + g.Ay * δyfy.data + g.Az * δzfz.data)
    nothing
end

### Flux divergences $\nabla \cdotp (\mathbf{u} Q)$
### $$\nabla \cdot (\mathbf{v} T) = \frac{1}{V} \left[ \delta_x (A_x u \overline{T}^x) + \delta_y (A_y v \overline{T}^y) + \delta_z (A_z w \overline{T}^z) \right]$$

In [ ]:
function div_flux(u, v, w, Q)
    flux_x = Aˣ .* u .* avgˣc2f(Q)
    flux_y = Aʸ .* v .* avgʸc2f(Q)
    flux_z = Aᶻ .* w .* avgᶻc2f(Q)

    # Imposing zero vertical flux through the top layer.
    @. flux_z[:, :, 1] = 0

    (1/V) .* (δˣf2c(flux_x) .+ δʸf2c(flux_y) .+ δᶻf2c(flux_z))
end

### Nonlinear advection terms $\mathbf{v} \cdot \nabla u$
### $$ \mathbf{v} \cdot \nabla u
    = \frac{1}{V} \left[
      \delta_x \left( \overline{A_x u}^x \overline{u}^x \right)
      + \delta_y \left( \overline{A_y v}^x \overline{u}^y \right)
      + \delta_z \left( \overline{A_z w}^x \overline{u}^z \right) \right]$$

In [ ]:
function ũ∇u(u, v, w)
    (1/V) .* (δˣc2f(avgˣf2c(Aˣ.*u) .* avgˣf2c(u)) + δʸc2f(avgˣf2c(Aʸ.*v) .* avgʸf2c(u)) + δᶻc2f(avgˣf2c(Aᶻ.*w) .* avgᶻf2c(u)))
end

function u∇u!(g::RegularCartesianGrid, ũ::VelocityFields, u∇u::FaceFieldX, tmp::TemporaryFields)
    u̅ˣ = tmp.fC1
    avgx!(g, ũ.u, u̅ˣ)

    uu = tmp.fC1
    @. uu.data = g.Ax * u̅ˣ.data^2

    u̅ʸ, v̅ˣ = tmp.fC2, tmp.fC3
    avgy!(g, ũ.u, u̅ʸ)
    avgx!(g, ũ.v, v̅ˣ)

    uv = tmp.fC2
    @. uv.data = g.Ay * u̅ʸ.data * v̅ˣ.data

    u̅ᶻ, w̅ˣ = tmp.fC3, tmp.fC4
    avgz!(g, ũ.u, u̅ᶻ)
    avgx!(g, ũ.w, w̅ˣ)

    uw = tmp.fC3
    @. uw.data = g.Az * u̅ᶻ.data * w̅ˣ.data

    ∂uu∂x, ∂uv∂y, ∂uw∂z = tmp.fFX, tmp.fFY, tmp.fFZ
    δx!(g, uu, ∂uu∂x)
    δy!(g, uv, ∂uv∂y)
    δz!(g, uw, ∂uw∂z)

    @. u∇u.data = (1/g.V) * (∂uu∂x.data + ∂uv∂y.data + ∂uw∂z.data)
    nothing
end

### Laplacian diffusion $\nabla\cdotp(\kappa\nabla Q)$
### $$\nabla \cdot (\kappa \nabla T) = \frac{1}{V} \nabla \cdot \left[ \kappa \left( A_x\delta_x T, A_y\delta_y T, A_z\delta_z T \right) \right]$$

In [ ]:
function κ∇²(Q)
    κ∇Q_x = κʰ .* δˣc2f(Q) ./ Δx
    κ∇Q_y = κʰ .* δʸc2f(Q) ./ Δy
    κ∇Q_z = κᵛ .* δᶻc2f(Q) ./ Δz
    div_f2c(κ∇Q_x, κ∇Q_y, κ∇Q_z)
end

### Viscous dissipation $\nabla\cdotp(\nu\nabla u)$
### $$\nabla \cdot (\nu \nabla u) = \frac{1}{V} \nabla \cdot \left[ \nu \left( \overline{A_x}^x \delta_x u, \overline{A_y}^y \delta_y u, \overline{A_z}^z \delta_z u \right) \right]$$

In [ ]:
# Example: Viscous dissipation for the u-velocity
function 𝜈ʰ∇²u(u)
    𝜈∇u_x = 𝜈ʰ .* δˣf2c(u) ./ Δx
    𝜈∇u_y = 𝜈ʰ .* δʸc2f(u) ./ Δy
    𝜈∇u_z = 𝜈ᵛ .* δᶻc2f(u) ./ Δz

    # div_c2f(𝜈∇u_x, 𝜈∇u_y, 𝜈∇u_z)
    (δˣc2f(Aˣ .* 𝜈∇u_x) + δʸf2c(Aʸ .* 𝜈∇u_y) + δᶻf2c(Aᶻ .* 𝜈∇u_z)) / V
end

## **Elliptic Poisson equation for the non-hydrostatic pressure**
### $$\nabla^2 \phi_{nh} = \nabla \cdotp \mathbf{G}_u = \mathscr{F}$$
* ### Actually solve for the vertical velocity needed to satisfy continuity $\nabla\cdotp\mathbf{u}=0$, which allows you to resolve small-scale motions and features.

## **Fourier-spectral solver for 3D Poisson equation with horizontally periodic and vertically Neumann boundary conditions**
### $$ \text{Step 1:} \quad\quad \hat{\mathscr{F}} = \text{DCT}_z\left[ \text{FFT}_{xy}(\mathscr{F}) \right] $$
### $$ \text{Step 2:} \quad\quad \hat{\phi}_{nh} = -\frac{\hat{\mathscr{F}}}{\mathbf{k}^2} $$
### $$ \text{Step 3:} \quad\quad \phi_{nh} = \text{IDCT}_z\left[ \text{IFFT}_{xy}(\hat{\phi}_{nh}) \right] $$
### where $\mathbf{k}^2 = k_x^2 + k_y^2 + k_z^2$ and
### $$
\begin{align}
k_x^2(i) &= 4\frac{N_x^2}{L_x^2}  \sin^2\left( \frac{(i-1)\pi}{N_x} \right), \quad i=1,2,\dots,N_x-1 \\
k_y^2(j) &= 4\frac{N_y^2}{L_y^2}  \sin^2\left( \frac{(j-1)\pi}{N_y} \right), \quad j=1,2,\dots,N_y-1 \\
k_z^2(k) &= 4\frac{N_z^2}{L_z^2}  \sin^2\left( \frac{(k-1)\pi}{2N_z} \right), \quad k=1,2,\dots,N_z-1
\end{align}
$$
---

In [ ]:
function solve_poisson_3d_mbc(f, Lx, Ly, Lz)
    Nx, Ny, Nz = size(f)  # Number of grid points (excluding the periodic end point).

    # Forward transform the real-valued source term.
    fh = FFTW.dct(FFTW.rfft(f, [1, 2]), 3)

    # Wavenumber indices.
    l1 = 0:Int(Nx/2)
    l2 = Int(-Nx/2 + 1):-1
    m1 = 0:Int(Ny/2)
    m2 = Int(-Ny/2 + 1):-1
    n1 = 0:Int(Nz/2)
    n2 = Int(-Nz/2 + 1):-1

    kx² = reshape((4/Δx^2) .* sin.( (π/Nx) .* cat(l1, l2, dims=1)).^2, (Nx, 1, 1))
    ky² = reshape((4/Δy^2) .* sin.( (π/Ny) .* cat(m1, m2, dims=1)).^2, (1, Ny, 1))
    kz² = reshape((4/Δz^2) .* sin.( (π/(2*Nz) .* 0:(Nz-1))).^2, (1, 1, Nz))

    k² = @. kx² + ky² + kz²

    ϕh = - fh ./ k²[1:Int(Nx/2 + 1), :, :]

    # Setting the DC/zero Fourier component to zero.
    ϕh[1, 1, 1] = 0

    # Take the inverse transform of the solution's Fourier coefficients.
    ϕ = FFTW.irfft(FFTW.idct(ϕh, 3), Nx, [1, 2])
end

## **Profiling results**

In [ ]:
using Pkg
cd("C:\\Users\\Ali\\Documents\\Git\\Oceananigans.jl\\")
Pkg.activate(".");

In [9]:
include("benchmark/benchmarks.jl")
run_benchmarks()

┌─────────────────────────────────────────────────────────────────────────────────────────────────────────────┐
│  BENCHMARKING OCEANANIGANS: T=Float64 (Nx, Ny, Nz) = (100, 100, 100)                                        │
├───────────────┬────────────┬──────────┬────────────┬────────────┬────────────┬────────────┬─────────┬───────┤
│ function name │   memory   │  allocs  │  min. time │  med. time │  mean time │  max. time │ samples │ evals │
├───────────────┼────────────┼──────────┼────────────┼────────────┼────────────┼────────────┼─────────┼───────┤
│     δx! (f2c) │    0 bytes │        0 │   1.278 ms │   1.742 ms │   2.187 ms │   9.619 ms │   10000 │     1 │
│     δx! (c2f) │    0 bytes │        0 │   1.894 ms │   2.870 ms │   3.289 ms │   9.092 ms │   10000 │     1 │
│     δy! (f2c) │    0 bytes │        0 │ 755.236 μs │   1.864 ms │   1.626 ms │   4.143 ms │   10000 │     1 │
│     δy! (c2f) │    0 bytes │        0 │ 770.188 μs │   1.921 ms │   1.643 ms │   3.783 ms │   10000 │ 

![](img/benchmarks.png)

## Back-of-the-envelope calculation for a $100\times100\times50$ grid
### Fortran (with conjugate-gradient solver): ~700 ms per time step
### Julia (with spectral solver): ~300 ms per time step

# Demo

In [1]:
using Pkg
cd("C:\\Users\\Ali\\Documents\\Git\\Oceananigans.jl\\")
Pkg.activate(".");

In [10]:
Pkg.test("Oceananigans")

   Testing Oceananigans
 Resolving package versions...
Test Summary: | Pass  Total
Grid          |    6      6
Test Summary: | Pass  Total
Fields        |  112    112
Test Summary: | Pass  Total
Operators     |   96     96
Test Summary:    | Pass  Total
Spectral solvers |   57     57
   Testing Oceananigans tests passed 


In [3]:
include("examples\\deep_convection.jl")

┌ Info: Ocean LES model parameters:
│ NumType: Float64
│ (Nˣ, Nʸ, Nᶻ) = (100, 100, 100) [#]
│ (Lˣ, Lʸ, Lᶻ) = (2000, 2000, 2000) [m]
│ (Δx, Δy, Δz) = (20.0, 20.0, 20.0) [m]
│ (Aˣ, Aʸ, Aᶻ) = (400.0, 400.0, 400.0) [m²]
│ V = 8000.0 [m³]
│ M = 8.216e6 [kg]
│ Nᵗ = 400 [s]
│ Δt = 30 [s]
└ @ Main C:\Users\Ali\Documents\Git\Oceananigans.jl\examples\deep_convection.jl:130
┌ Info: T₀[50, 50, 1] = 283 K
└ @ Main C:\Users\Ali\Documents\Git\Oceananigans.jl\examples\deep_convection.jl:178
┌ Info: Time: 150
│ uⁿ:   min=-3.67629e-05, max=3.67307e-05, mean=1.81006e-23, absmean=3.34074e-08, std=4.6774e-07
│ vⁿ:   min=-3.67196e-05, max=3.6742e-05, mean=-7.99049e-24, absmean=3.33989e-08, std=4.67268e-07
│ wⁿ:   min=-3.17763e-05, max=3.11308e-05, mean=-2.57652e-24, absmean=5.18006e-08, std=5.4707e-07
│ Tⁿ:   min=282.998, max=283, mean=283, absmean=283, std=8.13503e-05
│ Sⁿ:   min=35, max=35, mean=35, absmean=35, std=0
│ pʰʸ:  min=100714, max=2.00421e+07, mean=1.00714e+07, absmean=1.00714e+07, std=5.81445e+

In [4]:
using Interact, Plots, PyPlot
Plots.gr()

HTML{String}("    <script id=\"webio-setup-10223757398585738798\">\n    // Immediately-invoked-function-expression to avoid global variables.\n    (function() {\n        var warning_div = document.getElementById(\"webio-warning-4950671486289341766\");\n        var hide = function () {\n            var script = document.getElementById(\"webio-setup-10223757398585738798\");\n            var parent = script && script.parentElement;\n            var grandparent = parent && parent.parentElement;\n            if (grandparent) {\n                grandparent.style.display = \"none\";\n            }\n            warning_div.style.display = \"none\";\n        };\n        if (window.require && require.defined) {\n            // Jupyter notebook.\n            if (require.defined(\"nbextensions/webio/main\")\n                    || require.defined(\"/assetserver/28f5f30a59da434290b7f09bd0d0ddea74a5f34e-dist/main.js\")) {\n                // Extension already loaded.\n                hide();\n                return;\n            }\n            console.warn(\"Loading WebIO Jupyter extension on an ad-hoc basis. Consider enabling the WebIO nbextension for a stabler experience (this should happen automatically when building WebIO).\");\n            require([\"/assetserver/28f5f30a59da434290b7f09bd0d0ddea74a5f34e-dist/main.js\"], function (webIOModule) {\n                webIOModule.load_ipython_extension();\n            });\n            warning_div.innerHTML = \"<strong>Loading WebIO Jupyter extension on an ad-hoc basis. Consider enabling the WebIO nbextension for a stabler experience (this should happen automatically when building WebIO).</strong>\";\n        } else if (window.location.pathname.includes(\"/lab\")) {\n            // Guessing JupyterLab\n            warning_div.innerHTML = \"WebIO does not support JupyterLab yet.\";\n        }\n    })();\n    </script>\n    <div\n        id=\"webio-warning-4950671486289341766\"\n        class=\"output_text output_stderr\"\n        style=\"padding: 1em; font-weight: bold;\"\n    >\n        Unable to load WebIO. Please make sure WebIO works for your Jupyter client.\n        <!-- TODO: link to installation docs. -->\n    </div>\n")

Plots.GRBackend()

In [11]:
for j in [1, 10, 50]
    anim = @animate for tidx in 1:Int(Nᵗ/ΔR)
        Plots.heatmap(xC, zC, rotl90(RT[tidx,:,j,:]), color=:balance,
                title="delta T @ t=$(tidx*ΔR*Δt), j=$j")
    end
    js = lpad(j, 3, '0')
    mp4(anim, "T_j$js.mp4", fps = 10)
end

GKS: Rectangle definition is invalid in routine SET_WINDOW
GKS: Rectangle definition is invalid in routine CELLARRAY
origin outside current window
GKS: Rectangle definition is invalid in routine SET_WINDOW
GKS: Rectangle definition is invalid in routine CELLARRAY
origin outside current window
GKS: Rectangle definition is invalid in routine SET_WINDOW
GKS: Rectangle definition is invalid in routine CELLARRAY
origin outside current window
GKS: Rectangle definition is invalid in routine SET_WINDOW
GKS: Rectangle definition is invalid in routine CELLARRAY
origin outside current window
GKS: Rectangle definition is invalid in routine SET_WINDOW
GKS: Rectangle definition is invalid in routine CELLARRAY
origin outside current window
GKS: Rectangle definition is invalid in routine SET_WINDOW
GKS: Rectangle definition is invalid in routine CELLARRAY
origin outside current window
GKS: Rectangle definition is invalid in routine SET_WINDOW
GKS: Rectangle definition is invalid in routine CELLARRAY
o

In [6]:
for k in [1, 10, 50]
    anim = @animate for tidx in 1:Int(Nᵗ/ΔR)
        Plots.heatmap(xC, yC, RT[tidx,:,:,k] .- 283, color=:thermal,
            title="T change @ t=$(tidx*ΔR*Δt), k=$k")
    end
    ks = lpad(k, 3, '0')
    mp4(anim, "T_k$ks.mp4", fps = 10)
end

┌ Info: Saved animation to 
│   fn = C:\Users\Ali\Documents\Git\Oceananigans.jl\T_k001.mp4
└ @ Plots C:\Users\Ali\.julia\packages\Plots\rmogG\src\animation.jl:90
┌ Info: Saved animation to 
│   fn = C:\Users\Ali\Documents\Git\Oceananigans.jl\T_k010.mp4
└ @ Plots C:\Users\Ali\.julia\packages\Plots\rmogG\src\animation.jl:90
┌ Info: Saved animation to 
│   fn = C:\Users\Ali\Documents\Git\Oceananigans.jl\T_k050.mp4
└ @ Plots C:\Users\Ali\.julia\packages\Plots\rmogG\src\animation.jl:90


## **References**
### 1. Marshall, J., Hill, C., Perelman, L., Adcroft, A., **Hydrostatic, quasi-hydrostatic, and nonhydrostatic ocean modeling**, _Journal of Geophysical Research_ 102(C3), 5733-5752 (1997).
### 2. Marshall, J., Adcroft, A., Hill, C., Perelman, L., & Heisey, C., **A finite-volume, incompressible Navier Stokes model for studies of the ocean on parallel computers**, _Journal of Geophysical Research_ 102(C3), 5753-5766 (1997).
### 3. Alistair Adcroft, Jean-Michel Campin, Ed Doddridge, Stephanie Dutkiewicz, Constantinos Evangelinos, David Ferreira, Mick Follows, Gael Forget, Baylor Fox-Kemper, Patrick Heimbach, Chris Hill, Ed Hill, Helen Hill, Oliver Jahn, Jody Klymak, Martin Losch, John Marshall, Guillaume Maze, Matt Mazloff, Dimitris Menemenlis, Andrea Molod, and Jeff Scott. **MITgcm user manual**. (2008-2018)
### 4. Frigo, M., & Johnson, S. G., **The design and implementation of FFTW3**, _Proceedings of the IEEE_, 93(2), 216-231 (2005).